In [2]:
import pandas as pd
import numpy as np

In [10]:
import tensorflow as tf
#from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical

In [3]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, matthews_corrcoef
from sklearn.preprocessing import MinMaxScaler

In [89]:
from scipy.stats import ks_2samp

In [4]:
dados = pd.read_csv('base_tratada.csv')

In [5]:
dados.tail()

,MAU,TPCLIENT_2,LOCALIZA2_1.0,LOCALIZA2_2.0,"TPEMPREG2_(32.0, 130.0]","TPEMPREG2_(130.0, 628.0]",SEXO_M,ESTCIVIL2_1.0,ESTCIVIL2_2.0,SITRESID_P,"LIMITE2_(31.0, 118.0]","LIMITE2_(118.0, 99865.0]","TEMPORES2_(4.0, 70.0]",CEP2_1.0,CEP2_2.0,PROFISSAO2_1.0,PROFISSAO2_2.0,"ANONASCI2_(50.0, 63.0]","ANONASCI2_(63.0, 74.0]","ANONASCI2_(74.0, 87.0]"
7315,1,0,0,1,1,0,1,1,0,1,1,0,1,1,0,0,1,0,1,0
7316,0,0,0,1,1,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0
7317,0,0,0,1,0,1,1,1,0,1,1,0,0,1,0,1,0,1,0,0
7318,0,0,0,1,0,0,1,1,0,1,1,0,1,1,0,0,0,0,1,0
7319,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1


In [6]:
dados.shape

(7320, 20)

In [16]:
dados.iloc[:,1:19]

,TPCLIENT_2,LOCALIZA2_1.0,LOCALIZA2_2.0,"TPEMPREG2_(32.0, 130.0]","TPEMPREG2_(130.0, 628.0]",SEXO_M,ESTCIVIL2_1.0,ESTCIVIL2_2.0,SITRESID_P,"LIMITE2_(31.0, 118.0]","LIMITE2_(118.0, 99865.0]","TEMPORES2_(4.0, 70.0]",CEP2_1.0,CEP2_2.0,PROFISSAO2_1.0,PROFISSAO2_2.0,"ANONASCI2_(50.0, 63.0]","ANONASCI2_(63.0, 74.0]"
0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0
1,1,1,0,0,0,1,0,0,1,1,0,1,0,1,1,0,0,0
2,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0
3,0,1,0,0,1,0,0,1,1,1,0,1,0,1,0,0,0,0
4,0,1,0,1,0,0,0,1,1,1,0,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,0,0,1,1,0,1,1,0,1,1,0,1,1,0,0,1,0,1
7316,0,0,1,1,0,0,1,0,1,0,0,0,1,0,0,1,0,1
7317,0,0,1,0,1,1,1,0,1,1,0,0,1,0,1,0,1,0
7318,0,0,1,0,0,1,1,0,1,1,0,1,1,0,0,0,0,1


In [30]:
X_train, X_test, y_train, y_test = train_test_split(dados.iloc[:,1:20], dados['MAU'] , test_size=0.3 , random_state=42)

In [32]:
# Standardize features
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

# Reshape the input for LSTM
x_train_reshaped = x_train_scaled.reshape(x_train_scaled.shape[0], 1, x_train_scaled.shape[1])
x_test_reshaped = x_test_scaled.reshape(x_test_scaled.shape[0], 1, x_test_scaled.shape[1])

In [82]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(32, input_shape=(1, 19), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [83]:
# Train the model
model.fit(x_train_reshaped, y_train, epochs=10, batch_size=64)

Epoch 1/10
81/81 [==============================] - 1s 8ms/step - loss: 0.6457 - accuracy: 0.6673
Epoch 2/10
81/81 [==============================] - 1s 8ms/step - loss: 0.6082 - accuracy: 0.6920
Epoch 3/10
81/81 [==============================] - 1s 8ms/step - loss: 0.5962 - accuracy: 0.6926
Epoch 4/10
81/81 [==============================] - 1s 9ms/step - loss: 0.5931 - accuracy: 0.6963
Epoch 5/10
81/81 [==============================] - 1s 9ms/step - loss: 0.5882 - accuracy: 0.6991
Epoch 6/10
81/81 [==============================] - 1s 8ms/step - loss: 0.5890 - accuracy: 0.7012
Epoch 7/10
81/81 [==============================] - 1s 8ms/step - loss: 0.5831 - accuracy: 0.7045
Epoch 8/10
81/81 [==============================] - 1s 8ms/step - loss: 0.5834 - accuracy: 0.7082
Epoch 9/10
81/81 [==============================] - 1s 8ms/step - loss: 0.5804 - accuracy: 0.7065
Epoch 10/10
81/81 [==============================] - 1s 8ms/step - loss: 0.5811 - accuracy: 0.7082


In [84]:
test_loss, test_acc = model.evaluate(x_test_reshaped, y_test)
print('Test accuracy:', test_acc)

69/69 [==============================] - 0s 3ms/step - loss: 0.5592 - accuracy: 0.7104
Test accuracy: 0.7103825211524963


In [85]:
predictions = model.predict(x_test_reshaped)

69/69 [==============================] - 0s 2ms/step


In [86]:
roc_auc_score(y_test,  predictions.reshape(-1))

0.6947867685654326

In [102]:
y_preds_bin = pd.Series(predictions.reshape(-1)).apply(lambda x: 1 if x> 0.32 else 0)

In [103]:
matthews_corrcoef(y_test,  y_preds_bin )

0.2735723560575502

In [107]:
ks_2samp(predictions.reshape(-1)[y_test ==0],predictions.reshape(-1)[y_test ==1])

KstestResult(statistic=0.2995748020673245, pvalue=9.715744499131356e-37, statistic_location=0.31769577, statistic_sign=1)